# 03. 에이전트(Agent)

## 도구 호출 에이전트(Tool Calling Agent)

도구 호출을 사용하면 모델이 하나 이상의 도구(tool) 가 호출되어야 하는 시기를 감지하고 해당 도구에 전달해야 하는 입력 으로 전달할 수 있습니다.


In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("CH15-Agents")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agents


In [7]:
from langchain.tools import tool
from typing import List, Dict, Annotated
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

tools = [search_news, python_repl_tool]

### Agent 프롬프트 생성

- chat_history : 이전 대화 내용을 저장하는 변수 (멀티턴을 지원하지 않는다면, 생략 가능합니다.)
- agent_scratchpad : 에이전트가 임시로 저장하는 변수
- input : 사용자의 입력

In [8]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You are a helpful assistant."
            "Make sure to use the `search_news` tool for searching keyword related news."
         ),
        ("placeholder", "{chat_history}"), # 이전 대화 내용 저장하는 변수
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"), # 에이전트가 임시로 저장하는 변수
    ]
)

In [9]:
# 에이전트 생성

from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

### AgentExecutor

AgentExecutor는 도구를 사용하는 에이전트를 실행하는 클래스입니다.

주요 속성
- `agent`: 실행 루프의 각 단계에서 계획을 생성하고 행동을 결정하는 에이전트
- `tools`: 에이전트가 사용할 수 있는 유효한 도구 목록
- `return_intermediate_steps`: 최종 출력과 함께 에이전트의 중간 단계 경로를 반환할지 여부
- `max_iterations`: 실행 루프를 종료하기 전 최대 단계 수
- `max_execution_time`: 실행 루프에 소요될 수 있는 최대 시간
- `early_stopping_method`: 에이전트가 AgentFinish를 반환하지 않을 때 사용할 조기 종료 방법. ("force" or "generate")
    - "force" 는 시간 또는 반복 제한에 도달하여 중지되었다는 문자열을 반환합니다.
    - "generate" 는 에이전트의 LLM 체인을 마지막으로 한 번 호출하여 이전 단계에 따라 최종 답변을 생성합니다.
- `handle_parsing_errors`: 에이전트의 출력 파서에서 발생한 오류 처리 방법. (True, False, 또는 오류 처리 함수)
- `trim_intermediate_steps`: 중간 단계를 트리밍하는 방법. (-1 trim 하지 않음, 또는 트리밍 함수)

In [10]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools,
    verbose=True,
    max_iterations=10,
    max_execution_time=10,
    handle_parsing_errors=True,
)

# AgentExecutor 실행
result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

print("Agent 실행 결과:")
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMigwFBVV95cUxQZW1JVHBVNEtFM3ZSQi0zNlZGUVZDM3RHZGxZck8zTGxJZ1R6dUV2MWQzUFcxZFVSLS1ucU1jSXdJNzMzOGdQSGI1eG9NOEdvamg2bHBHN1FLaEdxSmpxRlk3bllZX19yQW5FZXhiU3lkNmQ4VEFVYlA3bm00SHFOVjRWONIBlwFBVV95cUxQa3hTVFZ6cnhPVmlZYVNNaW5uOVdVMFF2T1NRZm4waldnMWtaRnlaQ2xMQlB4MWVBV1lmbmJUcjhpamFBR3l3ZVZReXlRMUZ2ajBOVTh5LVFqdDF0X3R2eWV6RGwxTzZyeW9zWTdqTkdsdHUwTmNBeEMySHduOEZQYVlMbnB6ZVZ3blRFQ09KVG9zU3FKVWVn?oc=5', 'content': 'AI 투자 늘리는 MS “AI 데이터센터 구축에 연간 800억달러 투자할 것” - 조선일보'}, {'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTFA2aVFhLXlib0diQUNKeDJhUFkxSWhxODVNeEtkV3pZX281NHRzLUhscE5HNGwwZ0hleV8xbFZaTHN6bkpMVDRiOWJiZnFOdnJSRXNvNlV3?oc=5', 'content': '“하반기 증시 먹구름 걷힌다…AI빅테크 투자 패러다임은 ‘소프트웨어’”[투자360] - 헤럴드경제'}, {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTFBWdHFTOTVFR21GbHFzemVqNlJNOW9Ddy1QNGZEWFBsbVVfNGp1MlM0WHI5eXBuNUt6UzI0c2FLUFJ1bE0zRGlqX2hjWW9V

In [11]:
# 스트리밍 모드 실행
result = agent_executor.stream({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."}) 

for step in result:
    # 중간 단계 출력
    print(step)



> Entering new None chain...
{'actions': [ToolAgentAction(tool='search_news', tool_input={'query': 'AI 투자'}, log="\nInvoking: `search_news` with `{'query': 'AI 투자'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Z9TnpgbeHluLFoY4zpcuajJr', 'function': {'arguments': '{"query":"AI 투자"}', 'name': 'search_news'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b'}, id='run-71b6cf48-5ecb-4e12-9343-d44ad3a2341b', tool_calls=[{'name': 'search_news', 'args': {'query': 'AI 투자'}, 'id': 'call_Z9TnpgbeHluLFoY4zpcuajJr', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'search_news', 'args': '{"query":"AI 투자"}', 'id': 'call_Z9TnpgbeHluLFoY4zpcuajJr', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_Z9TnpgbeHluLFoY4zpcuajJr')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Z

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_ids):
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]


agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [13]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "안녕? 내 이름은 테디야!"},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    print(step)



> Entering new None chain...
안녕하세요, 테디! 만나서 반가워요. 어떻게 도와드릴까요?

> Finished chain.
{'output': '안녕하세요, 테디! 만나서 반가워요. 어떻게 도와드릴까요?', 'messages': [AIMessage(content='안녕하세요, 테디! 만나서 반가워요. 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={})]}


In [14]:
 
response = agent_with_chat_history.stream(
    {"input": "내 이름이 뭐라고?"},
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    for result in step:
        print(result)



> Entering new None chain...
당신의 이름은 테디입니다!

> Finished chain.
output
messages


In [17]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {
        "input": "내 이메일 주소는 teddy@teddynote.com 이야. 회사 이름은 테디노트 주식회사야."
    },
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    for result in step:
        print(result)



> Entering new None chain...
네, 테디! 당신의 이메일 주소는 teddy@teddynote.com이고, 회사 이름은 테디노트 주식회사입니다. 추가로 도움이 필요하시면 말씀해 주세요!

> Finished chain.
output
messages


In [18]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {
        "input": "최신 뉴스 5개를 검색해서 이메일의 본문으로 작성해줘. "
        "수신인에는 `셜리 상무님` 그리고, 발신인에는 내 인적정보를 적어줘."
        "정중한 어조로 작성하고, 메일의 시작과 끝에는 적절한 인사말과 맺음말을 적어줘."
    },
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    for result in step:
        print(result)



> Entering new None chain...
actions
messages

Invoking: `search_news` with `{'query': 'latest news'}`


[{'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTFB6b3FPREFINURLSy1pVUZDbXB4X3JwaW1fVGNCRks1RklJMDFNTWxCbURTbGMwU05kWVRRLU9xNnVFUkNPVFNLaUlTOThQaDRoQzlKQWtsQnVETnFqbEhGc2hIQnU2RERoS3c?oc=5', 'content': '인사이드 구글 - The Keyword'}, {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE0xUUFmMlVnRFBVbG94ZDhwMGd4SUlFcUoxNXVGeDJHQVp3QXhvZUdCVEdjb0ZvLXVfMXZrRHo2a3VIWlljV3E4aHh1RzhoaE56RmFVOW10SUhFNkpMTHdDODZRU1BERW51Rmc?oc=5', 'content': "[Breaking News] South Korean Court Rejects President Yoon's Appeal, Clearing Way for Presidential Office Raid - 한국면세뉴스"}, {'url': 'https://news.google.com/rss/articles/CBMiaEFVX3lxTE1RUEhYZHM4YU5RU1BUeE9udGl5LTI4QkIwdzREbUdEZnNmSGZ4NnFyMTNpVGNvRlQ1UERfU2RFZERZOVJ4S2Q3S2EwUjlXTUNSNkpxTWpIRlljVWNZVEpyd3BvdlFrb0hU0gFsQVVfeXFMT3FuQ1pPRVZFbEw5Ym9DYlBWbE9HQWlqTkRLUEFUUmlVcjkwQjlrNWktRXNEcXRKdFU4cXU2UjJOeEIxZEYwdmlJMllEU3V1Ul92ZW52eUt5SnktM25oS